In [ ]:
# the following notebook is mostly based on https://keras.io/guides/transfer_learning/

In [ ]:
%matplotlib inline

from IPython.display import display as d
from IPython.display import Image 

import pandas as pd
import numpy as np
import seaborn as sns
sns.set(rc={'figure.figsize':(12,12)})
sns.set_style("white")
import glob
import json
import os
import sys
import json
import shutil
import os
import sys
import multiprocessing
import shutil
from pprint import pprint

pool = multiprocessing.Pool(multiprocessing.cpu_count())


In [ ]:
!pip3 install keras tensorflow

In [ ]:
import tensorflow as tf

directory = "data/rule_of_thirds/"
batch_size = 32

def read_images(directory, subset, batch_size=32):
    return tf.keras.preprocessing.image_dataset_from_directory(
        directory, 
        labels='inferred', 
        label_mode='int',
        color_mode='rgb', 
        batch_size=batch_size, 
        image_size=(299, 299), 
        shuffle=True, 
        seed=42, 
        validation_split=0.1,  # 10% validation 
        subset=subset,
        interpolation='bilinear'
    )

training = read_images(directory, "training", batch_size)
print(training)

validation = read_images(directory, "validation", batch_size)
print(validation)



In [ ]:
import matplotlib.pyplot as plt

def overview_dataset(data):
    class_names = data.class_names

    plt.figure(figsize=(10, 10))
    for images, labels in data.take(1):
          for i in range(9):
                ax = plt.subplot(3, 3, i + 1)
                plt.imshow(images[i].numpy().astype("uint8"))
                plt.title(class_names[labels[i]])
                plt.axis("off")



In [ ]:
overview_dataset(training)

In [ ]:
overview_dataset(validation)

In [ ]:
import keras

base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(299, 299, 3))

# we cant do data_augmentation
# x = data_augmentation(inputs)  # Apply random data augmentation
x = inputs
# Pre-trained Xception weights requires that input be normalized
# from (0, 255) to a range (-1., +1.), the normalization layer
# does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 20
model.fit(training, epochs=epochs, validation_data=validation)